<a href="https://colab.research.google.com/github/manthan-mehta/ImageClassification/blob/master/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt

In [0]:
def rgb2gray(rgb):
  return np.dot(rgb[...,:3],[0.299,0.587,0.144])

In [0]:
from scipy.ndimage import uniform_filter
def hog_feature(im): #Histogram of Gradient(HOG)
  if im.ndim == 3:
    image = rgb2gray(im)
  else:
    image = np.atleast_2d(im)

  sx,sy = image.shape
  orientations = 9
  cx,cy = (8,8)

  gx = np.zeros(image.shape)
  gy = np.zeros(image.shape)
  gx[:,:-1] = np.diff(image,n=1,axis=1) #gradient in x-direction
  gy[:-1,:] = np.diff(image,n=1,axis=0) #gradient in y-direction

  grad_mag = np.sqrt(gx ** 2 + gy ** 2) # gradient magnitude
  grad_ori = np.arctan2(gy, (gx + 1e-15)) * (180 / np.pi) + 90 # gradient orientation

  n_cellsx = int(np.floor(sx / cx))  # number of cells in x
  n_cellsy = int(np.floor(sy / cy))  # number of cells in y

  #Final feature vector of HOG
  orientation_histogram = np.zeros((n_cellsx, n_cellsy, orientations))
  for i in range(orientations):
    # create new integral image for this orientation
    # isolate orientations in this range
    temp_ori = np.where(grad_ori < 180 / orientations * (i + 1),
                        grad_ori, 0)
    temp_ori = np.where(grad_ori >= 180 / orientations * i,
                        temp_ori, 0)
    
    # select magnitudes for those orientations
    cond2 = temp_ori > 0
    temp_mag = np.where(cond2, grad_mag, 0)
    orientation_histogram[:,:,i] = uniform_filter(temp_mag, size=(cx, cy))[int(cx/2)::cx, int(cy/2)::cy].T
  
  return orientation_histogram.ravel()

In [0]:
def color_histogram_hsv(im, nbin=10, xmin=0, xmax=255, normalized=True):
  # Compute color histogram for an image using hue.
  ndim = im.ndim
  bins = np.linspace(xmin, xmax, nbin+1)
  hsv = matplotlib.colors.rgb_to_hsv(im/xmax) * xmax
  imhist, bin_edges = np.histogram(hsv[:,:,0], bins=bins, density=normalized)
  imhist = imhist * np.diff(bin_edges)

  # return histogram
  return imhist

In [0]:
def extract_features(imgs, feature_fns, verbose=False):
  num_images = imgs.shape[0]
  if num_images == 0:
    return np.array([])

  # Use the first image to determine feature dimensions
  feature_dims = []
  first_image_features = []
  for feature_fn in feature_fns:
    feats = feature_fn(imgs[0].squeeze())
    assert len(feats.shape) == 1, 'Feature functions must be one-dimensional'
    feature_dims.append(feats.size)
    first_image_features.append(feats)

  # Now that we know the dimensions of the features, we can allocate a single
  # big array to store all features as columns.
  total_feature_dim = sum(feature_dims)
  imgs_features = np.zeros((num_images, total_feature_dim))
  imgs_features[0] = np.hstack(first_image_features).T

  # Extract features for the rest of the images.
  for i in range(1, num_images):
    idx = 0
    for feature_fn, feature_dim in zip(feature_fns, feature_dims):
      next_idx = idx + feature_dim
      imgs_features[i, idx:next_idx] = feature_fn(imgs[i].squeeze())
      idx = next_idx
    if verbose and i % 1000 == 0:
      print('Done extracting features for %d / %d images' % (i, num_images))

  return imgs_features

In [0]:
def load_cifar10(cifar10_folder_path,batch_id):
  import pickle
  with open(cifar10_folder_path + '/data_batch_' +str(batch_id),'rb') as fo:
    dict = pickle.load(fo,encoding='latin1')
  return dict

In [0]:
train = None
Y_train = []
for i in range(1,6):
  dict1 = load_cifar10('/content',1)
  if i==1:
    train = dict1['data']
  else:
    train = np.vstack((train,dict1['data']))
  Y_train += dict1['labels']

In [0]:
X_train = train.reshape((len(train), 3, 32, 32)).transpose(0, 2, 3, 1)
Y_train = np.asarray(Y_train)

In [0]:
import pickle
with open('/content/test_batch','rb') as fo:
    test_data = pickle.load(fo,encoding='latin1')
X_test = test_data['data'].reshape((len(test_data['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
Y_test = np.asarray(test_data['labels'])

In [0]:
num_training = 49000
num_validation = 1000
num_test = 1000

mask = list(range(num_training, num_training + num_validation))
X_val = X_train[mask]
Y_val = Y_train[mask]
mask = list(range(num_training))
X_train = X_train[mask]
Y_train = Y_train[mask]
mask = list(range(num_test))
X_test = X_test[mask]
Y_test = Y_test[mask]

In [19]:
num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000 / 49000 images
Done extracting features for 2000 / 49000 images
Done extracting features for 3000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 5000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 7000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 9000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 11000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 13000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 15000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 17000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 19000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extr

In [0]:
def softmax_loss(w,x,y,reg):
  dw = np.zeros_like(w)
  num_train = x.shape[0]
  loss = 0.0
  scores = x.dot(w)
  scores = scores - np.max(scores,axis=1,keepdims=True)
  sum_exp_scores = np.exp(scores).sum(axis=1,keepdims=True)
  softmax_matrix = np.exp(scores) / sum_exp_scores
  loss = np.sum(-np.log(softmax_matrix[np.arange(num_train),y]))
  softmax_matrix[np.arange(num_train),y] -= 1
  dw = (x.T).dot(softmax_matrix)

  loss /= num_train
  dw /= num_train

  # Regularization
  loss += reg * np.sum(w * w)
  dw += reg * 2 * w 
  return loss,dw 


In [0]:
#Check our feature vectors with our old classifiers
class Linear_Softmax(object):
  def __init__(self):
    self.W = None

  def train(self, X_train, Y_train, learning_rate, reg, num_iter, batch_size = 200, verbose = False):
    num_train,dim = X_train.shape
    num_class = np.max(Y_train) + 1
    if self.W is None:
      self.W = 0.001 * np.random.randn(dim,num_class)
    loss_history = []

    #stochastic gradient descent (batch wise)

    for it in range(num_iter):
      X_batch = None
      Y_batch = None
      
      batch_indices = np.random.choice(num_train, batch_size, replace=False)
      X_batch = X_train[batch_indices]
      Y_batch = Y_train[batch_indices]

      loss,grad = softmax_loss(self.W,X_batch,Y_batch,reg)
      loss_history.append(loss)

      self.W = self.W - learning_rate * grad
      if verbose and it % 100 == 0:
        print('iteration %d / %d: loss %f' % (it, num_iter, loss))

    return loss_history

  def predict(self,X):
    scores = X.dot(self.W)
    y_pred = np.zeros(X.shape[0])
    y_pred = np.argmax(scores,axis=1)
    return y_pred

In [24]:
learning_rates = [1e-3, 1e-2]
regularization_strengths = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
results = {}
best_val = -1
best_softmax = None
np.random.seed(0)
grid_search = [ (lr,reg) for lr in learning_rates for reg in regularization_strengths ]

for lr, reg in grid_search:
    # Create Softmax model
    softmax_model =Linear_Softmax()
    
    # Train phase
    softmax_model.train(X_train_feats, Y_train, learning_rate=lr, reg=reg, num_iter=2000,
            batch_size=200, verbose=False)
    y_train_pred = softmax_model.predict(X_train_feats)
    # Train accuracy
    train_accuracy = np.mean( y_train_pred == Y_train )
    
    # Validation phase
    y_val_pred = softmax_model.predict(X_val_feats)
    # Validation accuracy
    val_accuracy = np.mean( y_val_pred == Y_val )
    
    results[lr,reg] = (train_accuracy,val_accuracy)
    
    # Save best model
    if val_accuracy > best_val:
        best_val = val_accuracy
        best_softmax = softmax_model

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 1.000000e-03 reg 1.000000e-02 train accuracy: 0.462388 val accuracy: 0.463000
lr 1.000000e-03 reg 5.000000e-02 train accuracy: 0.459959 val accuracy: 0.462000
lr 1.000000e-03 reg 1.000000e-01 train accuracy: 0.459918 val accuracy: 0.464000
lr 1.000000e-03 reg 2.000000e-01 train accuracy: 0.456388 val accuracy: 0.462000
lr 1.000000e-03 reg 3.000000e-01 train accuracy: 0.453735 val accuracy: 0.462000
lr 1.000000e-03 reg 4.000000e-01 train accuracy: 0.453327 val accuracy: 0.462000
lr 1.000000e-03 reg 5.000000e-01 train accuracy: 0.452327 val accuracy: 0.451000
lr 1.000000e-03 reg 6.000000e-01 train accuracy: 0.448857 val accuracy: 0.456000
lr 1.000000e-03 reg 7.000000e-01 train accuracy: 0.444837 val accuracy: 0.443000
lr 1.000000e-02 reg 1.000000e-02 train accuracy: 0.527878 val accuracy: 0.519000
lr 1.000000e-02 reg 5.000000e-02 train accuracy: 0.514510 val accuracy: 0.509000
lr 1.000000e-02 reg 1.000000e-01 train accuracy: 0.500857 val accuracy: 0.508000
lr 1.000000e-02 reg 2.000000